# Building a Deep Q-Network with Keras

In [4]:
import gym
import numpy as np
import warnings
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque
import random

warnings.filterwarnings('ignore')
def warn(*args, **kwargs):
  pass
warnings.warn = warn

env = gym.make('CartPole-v1')

np.random.seed(42)
env.reset(seed=42)

(array([ 0.0273956 , -0.00611216,  0.03585979,  0.0197368 ], dtype=float32),
 {})

## Define the DQN model

In [3]:
def build_model(state_size, action_size):
  model = Sequential()
  model.add(Dense(24, input_dim=state_size, activation='relu'))
  model.add(Dense(24, activation='relu'))
  model.add(Dense(action_size, activation='linear'))
  model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
  return model

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
model = build_model(state_size, action_size)

## Implement the replay buffer

In [5]:
memory = deque(maxlen=2000)
def remember(state, action, reward, next_state, done):
  memory.append((state, action, reward, next_state, done))

## Implement the epsilon-greedy policy

In [7]:
epsilon = 1.0 # Exploration rage
epsilon_min = 0.01
epsilon_decay = 0.995

def act(state):
  if np.random.rand() <= epsilon:
    return random.randrange(action_size)
  q_values = model.predict(state)
  return np.argmax(q_values[0])

## Implement the Q-learning update

In [9]:
def replay(batch_size):
  global epsilon
  minibatch = random.sample(memory, batch_size)
  for state, action, reward, next_state, done in minibatch:
    target = reward
    if not done:
      target = reward + gamma * np.amax(model.predict(next_state)[0])
    target_f = model.predict(state)
    target_f[0][action] = target
    model.fit(state, target_f, epochs=1, verbose=0)
  if epsilon > epsilon_min:
    epsilon *=epsilon_decay

## Train the DQN

Train the DQN agent by interacting with the environment and updating the Q-values using the replay buffer.

In [11]:
episodes = 50
batch_size = 32
gamma = 0.95
 
for e in range(episodes):
  state = env.reset()
  if isinstance(state, tuple):  # Handle tuple output
    state = state[0]
  state = np.reshape(state, [1, state_size])

  for time in range(200):  # Max steps per episode
    # Choose action using epsilon-greedy policy
    action = act(state)

    # Perform action in the environment
    result = env.step(action)
    if len(result) == 4:  # Handle 4-value output
      next_state, reward, done, _ = result
    else:  # Handle 5-value output
      next_state, reward, done, _, _ = result

    if isinstance(next_state, tuple):  # Handle tuple next_state
      next_state = next_state[0]
    next_state = np.reshape(next_state, [1, state_size])

    # Store experience in memory
    remember(state, action, reward, next_state, done)

    # Update state
    state = next_state

    if done:  # If episode ends
      print(f"Episode: {e+1}/{episodes}, Score: {time}, Epsilon: {epsilon:.2}")
      break

  # Train the model using replay memory
  if len(memory) > batch_size:
    replay(batch_size)
 
env.close()

Episode: 1/50, Score: 14, Epsilon: 1.0
Episode: 2/50, Score: 20, Epsilon: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━

## Evaluate the performance

In [12]:
evaluation_episodes = 10
scores = []
 
for e in range(evaluation_episodes):
  state = env.reset()
  if isinstance(state, tuple):  # Handle tuple output
    state = state[0]
  state = np.reshape(state, [1, state_size])

  total_reward = 0
  for time in range(200):
    # Choose the greedy action
    action = np.argmax(model.predict(state)[0])

    # Perform action in the environment
    result = env.step(action)
    if len(result) == 4:  # Handle 4-value output
      next_state, reward, done, _ = result
    else:  # Handle 5-value output
      next_state, reward, done, _, _ = result

    if isinstance(next_state, tuple):  # Handle tuple next_state
      next_state = next_state[0]
    next_state = np.reshape(next_state, [1, state_size])

    state = next_state
    total_reward += reward

    if done:  # If episode ends
      print(f"Evaluation Episode: {e+1}/{evaluation_episodes}, Score: {time}, Total Reward: {total_reward}")
      scores.append(total_reward)
      break
 
print(f"Average Reward: {np.mean(scores):.2f}, Max Reward: {np.max(scores)}, Min Reward: {np.min(scores)}")
 
env.close()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Evaluation Episode: 1/10, Score: 9, Total Reward: 10.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Evaluation Episode: 2/10, Score: 9, Total Reward: 10.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━

## Exercise 1

Modify the reward structure to encourage the agent to keep the pole balanced longer.


1. Instead of just using the environment's reward, modify the reward function to include a penalty for large pole angles.
2. Update the reward calculation in the `train_dqn` function to discourage the agent from letting the pole deviate too far from the center.
3. Observe the effect on the agent's learning and episode length.


In [13]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir = 'dqn_sample_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
class1_train = os.path.join(train_dir, 'class1')
class2_train = os.path.join(train_dir, 'class2')
class1_val = os.path.join(val_dir, 'class1')
class2_val = os.path.join(val_dir, 'class2')

# Create directories if they do not exist
for dir_path in [train_dir, val_dir, class1_train, class2_train, class1_val, class2_val]:
  os.makedirs(dir_path, exist_ok=True)

print("Directory structure created. Add your images to these directories.")

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'sample_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'sample_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Function to modify the reward to encourage longer episodes
def modify_reward(reward, next_state):
    # Penalize large pole angles
    pole_angle = abs(next_state[2])  # Extract the pole angle from the state
    penalty = 1 if pole_angle > 0.1 else 0  # Apply penalty if angle is large
    return reward - penalty  # Adjust reward

Directory structure created. Add your images to these directories.
Found 16 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


## Exercise 2

Stop training early if the agent consistently reaches the maximum episode length.


1. Add an early stopping mechanism that stops training if the agent achieves a specified number of consecutive episodes with a length above a threshold.
2. Set the threshold as 195 steps for 100 consecutive episodes (for CartPole).
3. Print a message and stop training when this condition is met.


In [15]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'dqn_sample_data',  
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'sample_data',  
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


consecutive_success_threshold = 100
success_episode_length = 195
consecutive_success_count = 0
episode_lengths = []

# Example of training loop (this should be your actual loop)
for episode in range(1000):  # Replace with actual loop condition
  # Training logic goes here
  episode_length = 200
  episode_lengths.append(episode_length)
  
  # Early stopping check
  if len(episode_lengths) > consecutive_success_threshold and all(
    length >= success_episode_length for length in episode_lengths[-consecutive_success_threshold:]
  ):
    print("Early stopping: Agent consistently reaches max episode length.")
    break 

Found 0 images belonging to 2 classes.
Found 4 images belonging to 2 classes.
Early stopping: Agent consistently reaches max episode length.


## Exercise 3

Implement an epsilon decay schedule that switches from linear decay to exponential decay after a certain number of episodes.


1. Modify the epsilon decay strategy to start with a linear decay until a specific episode, then switch to an exponential decay.
2. Implement a check in the `choose_action` function to change the decay strategy after 100 episodes.
3. Observe and compare the agent’s performance.


In [16]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'sample_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'sample_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

def decay_epsilon(epsilon, episode, switch_episode=100):
  if episode < switch_episode:
    return max(epsilon - 0.01, 0.01)  # Linear decay
  else:
    return max(epsilon * 0.99, 0.01)  # Exponential decay

# Inside the training loop
epsilon = decay_epsilon(epsilon, e) 

Found 16 images belonging to 2 classes.
Found 4 images belonging to 2 classes.
